<a href="https://colab.research.google.com/github/dk-wei/advanced-pandas/blob/main/25_Pandas_TrIcks_%E5%86%B7%E9%97%A8%E6%8A%80%E5%B7%A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pycaret

     |████████████████████████████████| 264 kB 4.8 MB/s 
     |████████████████████████████████| 271 kB 48.8 MB/s 
     |████████████████████████████████| 108 kB 54.5 MB/s 
     |████████████████████████████████| 56 kB 3.3 MB/s 
     |████████████████████████████████| 167 kB 35.7 MB/s 
     |████████████████████████████████| 80 kB 8.5 MB/s 
     |████████████████████████████████| 2.0 MB 40.4 MB/s 
     |████████████████████████████████| 1.7 MB 41.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.3 MB 33.7 MB/s 
     |████████████████████████████████| 14.4 MB 60 kB/s 
     |████████████████████████████████| 248 kB 49.7 MB/s 
     |████████████████████████████████| 6.8 MB 67.5 MB/s 
     |████████████████████████████████| 3.1 MB 48.5 MB/s 
     |████████████████████████████████| 636 kB 52.1 MB/s 
     |███████████████████

In [36]:
import seaborn as sns
import pandas as pd
import numpy as np
from pycaret.datasets import get_data

In [4]:
# Load two datasets
diamonds = sns.load_dataset("diamonds")
diabetes = get_data('diabetes')
tips = sns.load_dataset("tips")

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


# ExcelWriter

write to `xlsx` file with different sheet name

In [6]:
# Write to the same excel file
with pd.ExcelWriter("data.xlsx") as writer:

    diamonds.to_excel(writer, sheet_name="diamonds")
    tips.to_excel(writer, sheet_name="tips")

# Pipe

pipe is one of the best functions for doing data cleaning in a concise, compact manner in Pandas. It allows you to chain multiple custom functions into a single operation.

For example, let’s say you have functions to `drop_duplicates`, `remove_outliers`, `encode_categoricals` that accept their own arguments. Here is how you apply all three in a single operation:

In [7]:
df_preped = (diamonds.pipe(drop_duplicates).
                      pipe(remove_outliers, ['price', 'carat', 'depth']).
                      pipe(encode_categoricals, ['cut', 'color', 'clarity'])
            )

NameError: ignored

# factorize

This function is a pandas alternative to Sklearn’s `LabelEncoder`:

In [8]:
# Mind the [0] at the end
diamonds["cut_enc"] = pd.factorize(diamonds["cut"])[0]

In [9]:
diamonds["cut_enc"].sample(5)

3018     0
39118    1
45776    0
26367    2
52579    0
Name: cut_enc, dtype: int64

Unlike LabelEncoder, factorize returns a tuple of two values: the encoded column and a list of the unique categories:


In [10]:
codes, unique = pd.factorize(diamonds["cut"], sort=True)

In [11]:
codes[:10]

array([0, 1, 3, 1, 3, 2, 2, 2, 4, 2])

In [12]:
unique

CategoricalIndex(['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], categories=['Ideal', 'Premium', 'Very Good', 'Good', 'Fair'], ordered=False, dtype='category')

# explode

Transform each element of a list-like to a row, replicating index values.



In [13]:
data = pd.Series([1, 6, 7, [46, 56, 49], 45, [15, 10, 12]]).to_frame("dirty")
data

,dirty
0,1
1,6
2,7
3,"[46, 56, 49]"
4,45
5,"[15, 10, 12]"


In [14]:
data.explode("dirty", ignore_index=True)

,dirty
0,1
1,6
2,7
3,46
4,56
5,49
6,45
7,15
8,10
9,12


In [15]:
data.explode("dirty")

,dirty
0,1
1,6
2,7
3,46
3,56
3,49
4,45
5,15
5,10
5,12


# squeezee

可以把一维的向量直接转为scaler，或者`Series`

In [16]:
subset = diamonds.loc[diamonds.index < 2, ["price"]]

In [17]:
subset.squeeze()

0    326
1    326
Name: price, dtype: int64

In [18]:
# squeeze过后可以直接通过位置索引
subset.squeeze()[1]

326

# between

A rather nifty function for boolean indexing numeric features within a range:

In [19]:
diamonds[diamonds["price"]\
      .between(3500, 3700, inclusive="neither")].sample(5)

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
4136,0.95,Very Good,I,SI1,61.0,61.0,3544,6.29,6.37,3.86,3
4647,0.90,Good,I,IF,63.7,64.0,3669,6.07,6.02,3.85,2
4545,1.00,Premium,G,SI2,63.0,56.0,3640,6.37,6.33,4.00,1
4790,1.00,Good,E,SI1,57.6,65.0,3696,6.47,6.44,3.72,2
4209,0.72,Premium,E,VVS1,62.9,59.0,3566,5.75,5.67,3.59,1


# T

transpose

In [20]:
diamonds.describe().T

,count,mean,std,min,25%,50%,75%,max
carat,53940.0,0.797940,0.474011,0.2,0.40,0.70,1.04,5.01
depth,53940.0,61.749405,1.432621,43.0,61.00,61.80,62.50,79.00
table,53940.0,57.457184,2.234491,43.0,56.00,57.00,59.00,95.00
price,53940.0,3932.799722,3989.439738,326.0,950.00,2401.00,5324.25,18823.00
x,53940.0,5.731157,1.121761,0.0,4.71,5.70,6.54,10.74
y,53940.0,5.734526,1.142135,0.0,4.72,5.71,6.54,58.90
z,53940.0,3.538734,0.705699,0.0,2.91,3.53,4.04,31.80
cut_enc,53940.0,1.228940,1.265976,0.0,0.00,1.00,3.00,4.00


# Style

In [21]:
diabetes.describe().T.drop("count", axis=1)\
.style.highlight_max()
               #  .style.highlight_max(color="darkred")

,mean,std,min,25%,50%,75%,max
Number of times pregnant,3.845052,3.369578,0.000000,1.000000,3.000000,6.000000,17.000000
Plasma glucose concentration a 2 hours in an oral glucose tolerance test,120.894531,31.972618,0.000000,99.000000,117.000000,140.250000,199.000000
Diastolic blood pressure (mm Hg),69.105469,19.355807,0.000000,62.000000,72.000000,80.000000,122.000000
Triceps skin fold thickness (mm),20.536458,15.952218,0.000000,0.000000,23.000000,32.000000,99.000000
2-Hour serum insulin (mu U/ml),79.799479,115.244002,0.000000,0.000000,30.500000,127.250000,846.000000
Body mass index (weight in kg/(height in m)^2),31.992578,7.884160,0.000000,27.300000,32.000000,36.600000,67.100000
Diabetes pedigree function,0.471876,0.331329,0.078000,0.243750,0.372500,0.626250,2.420000
Age (years),33.240885,11.760232,21.000000,24.000000,29.000000,41.000000,81.000000
Class variable,0.348958,0.476951,0.000000,0.000000,0.000000,1.000000,1.000000


Above, we are highlighting cells that hold the maximum value of a column. Another cool styler is `background_gradient` which can give columns a gradient background color based on their values:

In [22]:
diabetes.describe().T.drop("count", axis=1).style.background_gradient(
    subset=["mean", "50%"], cmap="Reds"
)

,mean,std,min,25%,50%,75%,max
Number of times pregnant,3.845052,3.369578,0.000000,1.000000,3.000000,6.000000,17.000000
Plasma glucose concentration a 2 hours in an oral glucose tolerance test,120.894531,31.972618,0.000000,99.000000,117.000000,140.250000,199.000000
Diastolic blood pressure (mm Hg),69.105469,19.355807,0.000000,62.000000,72.000000,80.000000,122.000000
Triceps skin fold thickness (mm),20.536458,15.952218,0.000000,0.000000,23.000000,32.000000,99.000000
2-Hour serum insulin (mu U/ml),79.799479,115.244002,0.000000,0.000000,30.500000,127.250000,846.000000
Body mass index (weight in kg/(height in m)^2),31.992578,7.884160,0.000000,27.300000,32.000000,36.600000,67.100000
Diabetes pedigree function,0.471876,0.331329,0.078000,0.243750,0.372500,0.626250,2.420000
Age (years),33.240885,11.760232,21.000000,24.000000,29.000000,41.000000,81.000000
Class variable,0.348958,0.476951,0.000000,0.000000,0.000000,1.000000,1.000000


# 

In [23]:
dir(pd.options)

['compute', 'display', 'io', 'mode', 'plotting']

In [24]:
dir(pd.options.display)

['chop_threshold',
 'colheader_justify',
 'column_space',
 'date_dayfirst',
 'date_yearfirst',
 'encoding',
 'expand_frame_repr',
 'float_format',
 'html',
 'large_repr',
 'latex',
 'max_categories',
 'max_columns',
 'max_colwidth',
 'max_info_columns',
 'max_info_rows',
 'max_rows',
 'max_seq_items',
 'memory_usage',
 'min_rows',
 'multi_sparse',
 'notebook_repr_html',
 'pprint_nest_depth',
 'precision',
 'show_dimensions',
 'unicode',
 'width']

In [25]:
pd.options.display.max_columns

0

There are many options under `display` but I mostly use `max_columns` and precision:

In [26]:
 # Remove the limit to display the number of cols
pd.options.display.max_columns = None

# Only show 5 numbers after the decimal
pd.options.display.precision = 5  # gets rid of scientific notation

# convert_dtypes

自动convert object to string，虽然我也不知道为啥不好使

In [34]:
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,0
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,1
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,2
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,1
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,2


In [31]:
diamonds.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
price         int64
x           float64
y           float64
z           float64
cut_enc       int64
dtype: object

In [32]:
diamonds.convert_dtypes().dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
price         Int64
x           float64
y           float64
z           float64
cut_enc       Int64
dtype: object

# select_dtypes

选择type为number的column，当然也可以选择type不为number的column

In [37]:
# Choose only numerical columns
diamonds.select_dtypes(include=np.number).head()

,carat,depth,table,price,x,y,z,cut_enc
0,0.23,61.5,55.0,326,3.95,3.98,2.43,0
1,0.21,59.8,61.0,326,3.89,3.84,2.31,1
2,0.23,56.9,65.0,327,4.05,4.07,2.31,2
3,0.29,62.4,58.0,334,4.20,4.23,2.63,1
4,0.31,63.3,58.0,335,4.34,4.35,2.75,2


In [38]:
# Exclude numerical columns
diamonds.select_dtypes(exclude=np.number).head()

,cut,color,clarity
0,Ideal,E,SI2
1,Premium,E,SI1
2,Good,E,VS1
3,Premium,I,VS2
4,Good,J,SI2


# mask

让你迅速的将不符合条件的值转变为其他值

In [39]:
# Create sample data
ages = pd.Series([55, 52, 50, 66, 57, 59, 49, 60]).to_frame("ages")

ages

,ages
0,55
1,52
2,50
3,66
4,57
5,59
6,49
7,60


In [40]:
ages.mask(cond=~ages["ages"].between(50, 60), other=np.nan)

,ages
0,55.0
1,52.0
2,50.0
3,NaN
4,57.0
5,59.0
6,NaN
7,60.0


# nlargest & nsmallest

In [41]:
diamonds.nlargest(5, "price")

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
27749,2.29,Premium,I,VS2,60.8,60.0,18823,8.50,8.47,5.16,1
27748,2.00,Very Good,G,SI1,63.5,56.0,18818,7.90,7.97,5.04,3
27747,1.51,Ideal,G,IF,61.7,55.0,18806,7.37,7.41,4.56,0
27746,2.07,Ideal,G,SI2,62.5,55.0,18804,8.20,8.13,5.11,0
27745,2.00,Very Good,H,SI1,62.8,57.0,18803,7.95,8.00,5.01,3


In [42]:
diamonds.nsmallest(5, "price")

,carat,cut,color,clarity,depth,table,price,x,y,z,cut_enc
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,0
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,1
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,2
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,1
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,2


# idxmax & idxmin

In [45]:
diamonds.price.idxmax()

27749

In [47]:
diamonds.price.idxmin()

0

# value_counts & dropna = False

In [50]:
diamonds["color"].value_counts(dropna=False, normalize=True)

G    0.20934
E    0.18163
F    0.17690
H    0.15395
D    0.12560
I    0.10052
J    0.05206
Name: color, dtype: float64

# clip

In [52]:
# Create sample data
ages = pd.Series([55, 52, 50, 66, 57, 59, 49, 60]).to_frame("ages")

ages

,ages
0,55
1,52
2,50
3,66
4,57
5,59
6,49
7,60


In [54]:
ages.clip(50, 60)

,ages
0,55
1,52
2,50
3,60
4,57
5,59
6,50
7,60


# at_time & between_time

In [56]:
index = pd.date_range("2021-08-01", periods=100, freq="H")
data = pd.DataFrame({"col": list(range(100))}, index=index)

data.head()

,col
2021-08-01 00:00:00,0
2021-08-01 01:00:00,1
2021-08-01 02:00:00,2
2021-08-01 03:00:00,3
2021-08-01 04:00:00,4


In [57]:
data.at_time("15:00")

,col
2021-08-01 15:00:00,15
2021-08-02 15:00:00,39
2021-08-03 15:00:00,63
2021-08-04 15:00:00,87


In [58]:
data.between_time("09:45", "12:00")

,col
2021-08-01 10:00:00,10
2021-08-01 11:00:00,11
2021-08-01 12:00:00,12
2021-08-02 10:00:00,34
2021-08-02 11:00:00,35
2021-08-02 12:00:00,36
2021-08-03 10:00:00,58
2021-08-03 11:00:00,59
2021-08-03 12:00:00,60
2021-08-04 10:00:00,82


# bdate_range

In [59]:
series = pd.bdate_range("2021-01-01", "2021-01-31")  # A period of one month

In [60]:
len(series)

21

In [61]:
series

DatetimeIndex(['2021-01-01', '2021-01-04', '2021-01-05', '2021-01-06',
               '2021-01-07', '2021-01-08', '2021-01-11', '2021-01-12',
               '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-18',
               '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22',
               '2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28',
               '2021-01-29'],
              dtype='datetime64[ns]', freq='B')

# autocorr

In [62]:
time_series = tips[["tip"]]
time_series["lag_1"] = time_series["tip"].shift(1)
time_series["lag_2"] = time_series["tip"].shift(2)
time_series["lag_3"] = time_series["tip"].shift(3)
time_series["lag_4"] = time_series["tip"].shift(4)
# time_series['lag_k'] = time_series['tip'].shift(k)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [63]:
time_series.head()

,tip,lag_1,lag_2,lag_3,lag_4
0,1.01,NaN,NaN,NaN,NaN
1,1.66,1.01,NaN,NaN,NaN
2,3.50,1.66,1.01,NaN,NaN
3,3.31,3.50,1.66,1.01,NaN
4,3.61,3.31,3.50,1.66,1.01


In [64]:
time_series["tip"].autocorr(lag=8)

0.07475238789967077

# hasnans

In [65]:
series = pd.Series([2, 4, 6, "sadf", np.nan])

series.hasnans

True

# at and iat

In [66]:
# [index, label]
diamonds.at[234, "cut"]

'Ideal'

In [67]:
diamonds.iat[1564, 4]

61.2

In [68]:
diamonds.at[16541, "price"] = 10000

# argsort

得到sort之后的indices

In [69]:
tips.reset_index(inplace=True, drop=True)

sort_idx = tips["total_bill"].argsort(kind="mergesort")

# Now, sort `tips` based on total_bill
tips.iloc[sort_idx].head()

,total_bill,tip,sex,smoker,day,time,size
67,3.07,1.00,Female,Yes,Sat,Dinner,1
92,5.75,1.00,Female,Yes,Fri,Dinner,2
111,7.25,1.00,Female,No,Sat,Dinner,1
172,7.25,5.15,Male,Yes,Sun,Dinner,2
149,7.51,2.00,Male,No,Thur,Lunch,2


In [70]:
sort_idx

0       67
1       92
2      111
3      172
4      149
      ... 
239    182
240    156
241     59
242    212
243    170
Name: total_bill, Length: 244, dtype: int64

# cat

In [71]:
diamonds.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
price         int64
x           float64
y           float64
z           float64
cut_enc       int64
dtype: object

In [74]:
diamonds["cut"].cat.categories

In [75]:
diamonds["new_cuts"] = diamonds["cut"].cat.rename_categories(list("ABCDE"))

In [76]:
diamonds["new_cuts"].cat.categories

Index(['A', 'B', 'C', 'D', 'E'], dtype='object')

# GroupBy.nth

返回grouping之后，每个group的第n个row

In [77]:
diamonds.groupby("cut").nth(5)

,carat,color,clarity,depth,table,price,x,y,z,cut_enc,new_cuts
cut,,,,,,,,,,,
Ideal,0.33,I,SI2,61.2,56.0,403,4.49,4.50,2.75,0,A
Premium,0.24,I,VS1,62.5,57.0,355,3.97,3.94,2.47,1,B
Very Good,0.23,E,VS2,63.8,55.0,352,3.85,3.92,2.48,3,C
Good,0.30,I,SI2,63.3,56.0,351,4.26,4.30,2.71,2,D
Fair,0.91,H,SI2,64.4,57.0,2763,6.11,6.09,3.93,4,E


In [78]:
diamonds.groupby("cut").nth(1)

,carat,color,clarity,depth,table,price,x,y,z,cut_enc,new_cuts
cut,,,,,,,,,,,
Ideal,0.23,J,VS1,62.8,56.0,340,3.93,3.90,2.46,0,A
Premium,0.29,I,VS2,62.4,58.0,334,4.20,4.23,2.63,1,B
Very Good,0.24,I,VVS1,62.3,57.0,336,3.95,3.98,2.47,3,C
Good,0.31,J,SI2,63.3,58.0,335,4.34,4.35,2.75,2,D
Fair,0.86,E,SI2,55.1,69.0,2757,6.45,6.33,3.52,4,E
